# CityBikes

Send a request to CityBikes for the city of your choice. 

In [2]:
import requests
import pandas as pd
import re

In [3]:
id = "capital-bikeshare"
url = "http://api.citybik.es/v2/networks/" + id
result = requests.get(url)
data = result.json()

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [4]:
stationjson = data['network']['stations']
stationjson

[{'id': '00082598b29c8a9211c1301ee0205a67',
  'name': '17th St & Independence Ave SW',
  'latitude': 38.888097,
  'longitude': -77.038325,
  'timestamp': '2024-10-20T03:40:10.965688Z',
  'free_bikes': 2,
  'empty_slots': 14,
  'extra': {'uid': '082623bf-1f3f-11e7-bf6b-3863bb334450',
   'renting': 1,
   'returning': 1,
   'last_updated': 1729395578,
   'has_ebikes': True,
   'ebikes': 2,
   'payment': ['key', 'creditcard'],
   'payment-terminal': True,
   'slots': 19,
   'rental_uris': {'android': 'https://dc.lft.to/lastmile_qr_scan',
    'ios': 'https://dc.lft.to/lastmile_qr_scan'}}},
 {'id': '003bcd1cd2030b68b53b8ed2b3415f5a',
  'name': '7th & Monroe St NE',
  'latitude': 38.932242581555066,
  'longitude': -76.9961330294609,
  'timestamp': '2024-10-20T03:40:10.965067Z',
  'free_bikes': 5,
  'empty_slots': 9,
  'extra': {'uid': '066fd95e-0d37-4eea-a027-88c04277bb9d',
   'renting': 1,
   'returning': 1,
   'last_updated': 1729395519,
   'has_ebikes': True,
   'ebikes': 1,
   'payment': 

Put your parsed results into a DataFrame.

In [9]:
df = pd.DataFrame(stationjson, columns=['name','latitude','longitude','free_bikes','empty_slots','extra'])
df['total_bike_cap'] = df['empty_slots'] + df['free_bikes']

extra_df = pd.json_normalize(df['extra'])
df.drop(columns=['extra'], inplace=True)
df = pd.concat([df, extra_df], axis=1)


Filtering down to a reasonable amount of stations for api calls. Removing any stations with digits in their name then taking a random 75. 

In [16]:
# decided to filter out stations with numbers because those would be in the city and my yelp and foursquare results would be a lot.
regex = r'\d'
df_filtered = df[~df['name'].str.contains(regex, na=False)]
# creating a random sample of 50 stations from the list
df_sampled = df_filtered.sample(n=75, random_state=42)
df_sampled.head()

,name,latitude,longitude,free_bikes,empty_slots,total_bike_cap,uid,renting,returning,last_updated,has_ebikes,ebikes,payment,payment-terminal,slots,rental_uris.android,rental_uris.ios
670,Fairfax Dr & Wilson Blvd,38.885801,-77.097745,5,6,11,08251ded-1f3f-11e7-bf6b-3863bb334450,1,1,1729395530,True,3,"[key, creditcard]",True,12,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan
761,South St & University Dr,38.844249,-77.305336,11,1,12,1877158937567185816,1,1,1729395487,True,0,"[key, creditcard]",True,12,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan
295,White Oak Rec Center,39.046143,-76.975723,6,9,15,2213781a-5c69-43eb-9b33-346d61e7e1ba,1,1,1729395477,True,0,"[key, creditcard]",True,15,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan
716,Fessenden St & Wisconsin Ave NW,38.954812,-77.082426,1,14,15,08254529-1f3f-11e7-bf6b-3863bb334450,1,1,1729395578,True,0,"[key, creditcard]",True,15,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan
480,Fenton St & Gist Ave,38.989724,-77.023854,6,5,11,0825ac97-1f3f-11e7-bf6b-3863bb334450,1,1,1729395538,True,5,"[key, creditcard]",True,11,https://dc.lft.to/lastmile_qr_scan,https://dc.lft.to/lastmile_qr_scan


In [17]:
df_sampled.to_csv('../data/bikedata.csv', index=False)